<a href="https://colab.research.google.com/github/NadiaSouza1304/Hack_DS_01/blob/previous_application/Previous_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Criação de variáveis a partir da tabela Previous_application

## Preparação do Ambiente

### Habilitar o Spark para o colab.


In [12]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

### Importar Bibliotecas


In [13]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [14]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

### Mapeando o Google Drive, para possibilitar o acesso aos arquivos de leitura

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Ler os arquivos e disponibilizar view para manipular com SQL.

O arquivo previous_application.csv: Contém informações sobre aplicações de empréstimo anteriores de um cliente na PoD Bank.

In [16]:
df_previous_application = spark.read.csv("/content/gdrive/MyDrive/HD/CURSOS/Ciência - Análise de Dados/Pod Academy/Formação em Big Data e Analytics/Hackathon - Ciência de Dados/11 - Hackaton/dados/previous_application.csv", header=True,inferSchema=True)
df_previous_application.createOrReplaceTempView("df_previous_application")

##Criação das variáveis - Tabela previous_application

Verificar schema da tabela (metadados).

In [17]:
df_previous_application.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_APPLICATION: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_DOWN_PAYMENT: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- WEEKDAY_APPR_PROCESS_START: string (nullable = true)
 |-- HOUR_APPR_PROCESS_START: integer (nullable = true)
 |-- FLAG_LAST_APPL_PER_CONTRACT: string (nullable = true)
 |-- NFLAG_LAST_APPL_IN_DAY: integer (nullable = true)
 |-- RATE_DOWN_PAYMENT: double (nullable = true)
 |-- RATE_INTEREST_PRIMARY: double (nullable = true)
 |-- RATE_INTEREST_PRIVILEGED: double (nullable = true)
 |-- NAME_CASH_LOAN_PURPOSE: string (nullable = true)
 |-- NAME_CONTRACT_STATUS: string (nullable = true)
 |-- DAYS_DECISION: integer (nullable = true)
 |-- NAME_PAYMENT_TYPE: string (nullable = true)
 |-- CODE_REJECT_REASON: string (nullable = t

Mostrar tabela

In [18]:
df_previous_application.show(10,truncate=False)

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURP

Contagem de linhas totais (Considerando as repetidas)

In [19]:
df_previous_application.count()

1670214

Contagem de linhas distintas

In [20]:
spark.sql("""
SELECT
COUNT(DISTINCT(SK_ID_CURR)) AS Quantidade_SK_ID
FROM
df_previous_application
""").show()

+----------------+
|Quantidade_SK_ID|
+----------------+
|          338857|
+----------------+



### Analisando os dados categóricos da tabela previous_application:

- NAME_CONTRACT_TYPE
- WEEKDAY_APPR_PROCESS_START
- FLAG_LAST_APPL_PER_CONTRACT
- NAME_CASH_LOAN_PURPOSE
- NAME_CONTRACT_STATUS
- NAME_PAYMENT_TYPE
- CODE_REJECT_REASON
- NAME_TYPE_SUITE
- NAME_CLIENT_TYPE
- NAME_GOODS_CATEGORY
- NAME_PORTFOLIO
- NAME_PRODUCT_TYPE
- CHANNEL_TYPE
- NAME_SELLER_INDUSTRY
- NAME_YIELD_GROUP
- PRODUCT_COMBINATION

In [21]:
spark.sql("""
SELECT
SK_ID_CURR,
NAME_CONTRACT_TYPE,
WEEKDAY_APPR_PROCESS_START,
FLAG_LAST_APPL_PER_CONTRACT,
NAME_CASH_LOAN_PURPOSE,
NAME_CONTRACT_STATUS,
NAME_PAYMENT_TYPE,
CODE_REJECT_REASON,
NAME_TYPE_SUITE,
NAME_CLIENT_TYPE,
NAME_GOODS_CATEGORY,
NAME_PORTFOLIO,
NAME_PRODUCT_TYPE,
CHANNEL_TYPE,
NAME_SELLER_INDUSTRY,
NAME_YIELD_GROUP,
PRODUCT_COMBINATION
FROM
df_previous_application
WHERE SK_ID_CURR = '100003'
""").show()

+----------+------------------+--------------------------+---------------------------+----------------------+--------------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+--------------------+----------------+--------------------+
|SK_ID_CURR|NAME_CONTRACT_TYPE|WEEKDAY_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NAME_CASH_LOAN_PURPOSE|NAME_CONTRACT_STATUS|   NAME_PAYMENT_TYPE|CODE_REJECT_REASON|NAME_TYPE_SUITE|NAME_CLIENT_TYPE| NAME_GOODS_CATEGORY|NAME_PORTFOLIO|NAME_PRODUCT_TYPE|        CHANNEL_TYPE|NAME_SELLER_INDUSTRY|NAME_YIELD_GROUP| PRODUCT_COMBINATION|
+----------+------------------+--------------------------+---------------------------+----------------------+--------------------+--------------------+------------------+---------------+----------------+--------------------+--------------+-----------------+--------------------+--------------------+----------------+--------

Analisando a tabela acima vemos que cada coluna categórica possui domínios variados.

###Identificando todos domínios das variáveis categóricas da tabela previous_application:

- NAME_CONTRACT_TYPE
- WEEKDAY_APPR_PROCESS_START
- FLAG_LAST_APPL_PER_CONTRACT
- NAME_CASH_LOAN_PURPOSE
- NAME_CONTRACT_STATUS
- NAME_PAYMENT_TYPE
- CODE_REJECT_REASON
- NAME_TYPE_SUITE
- NAME_CLIENT_TYPE
- NAME_GOODS_CATEGORY
- NAME_PORTFOLIO
- NAME_PRODUCT_TYPE
- CHANNEL_TYPE
- NAME_SELLER_INDUSTRY
- NAME_YIELD_GROUP
- PRODUCT_COMBINATION

In [22]:
spark.sql("""
SELECT
NAME_CONTRACT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CONTRACT_TYPE
ORDER BY Count(*)
""").show()

+------------------+--------+
|NAME_CONTRACT_TYPE|count(1)|
+------------------+--------+
|               XNA|     346|
|   Revolving loans|  193164|
|    Consumer loans|  729151|
|        Cash loans|  747553|
+------------------+--------+



In [23]:
spark.sql("""
SELECT
WEEKDAY_APPR_PROCESS_START,
Count(*)
FROM
df_previous_application
GROUP BY WEEKDAY_APPR_PROCESS_START
ORDER BY Count(*)
""").show()

+--------------------------+--------+
|WEEKDAY_APPR_PROCESS_START|count(1)|
+--------------------------+--------+
|                    SUNDAY|  164751|
|                  SATURDAY|  240631|
|                  THURSDAY|  249099|
|                    FRIDAY|  252048|
|                    MONDAY|  253557|
|                 WEDNESDAY|  255010|
|                   TUESDAY|  255118|
+--------------------------+--------+



In [24]:
spark.sql("""
SELECT
FLAG_LAST_APPL_PER_CONTRACT,
Count(*)
FROM
df_previous_application
GROUP BY FLAG_LAST_APPL_PER_CONTRACT
ORDER BY Count(*)
""").show()

+---------------------------+--------+
|FLAG_LAST_APPL_PER_CONTRACT|count(1)|
+---------------------------+--------+
|                          N|    8475|
|                          Y| 1661739|
+---------------------------+--------+



In [25]:
spark.sql("""
SELECT
NAME_CASH_LOAN_PURPOSE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CASH_LOAN_PURPOSE
ORDER BY Count(*)
""").show(truncate=False)

+--------------------------------+--------+
|NAME_CASH_LOAN_PURPOSE          |count(1)|
+--------------------------------+--------+
|Refusal to name the goal        |15      |
|Money for a third person        |25      |
|Hobby                           |55      |
|Buying a garage                 |136     |
|Gasification / water supply     |300     |
|Business development            |426     |
|Buying a holiday home / land    |533     |
|Furniture                       |749     |
|Car repairs                     |797     |
|Buying a home                   |865     |
|Wedding / gift / holiday        |962     |
|Buying a new car                |1012    |
|Purchase of electronic equipment|1061    |
|Journey                         |1239    |
|Education                       |1573    |
|Payments on other loans         |1931    |
|Medicine                        |2174    |
|Everyday expenses               |2416    |
|Building a house or an annex    |2693    |
|Buying a used car              

In [26]:
spark.sql("""
SELECT
NAME_CONTRACT_STATUS,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CONTRACT_STATUS
ORDER BY Count(*)
""").show()

+--------------------+--------+
|NAME_CONTRACT_STATUS|count(1)|
+--------------------+--------+
|        Unused offer|   26436|
|             Refused|  290678|
|            Canceled|  316319|
|            Approved| 1036781|
+--------------------+--------+



In [27]:
spark.sql("""
SELECT
NAME_PAYMENT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_PAYMENT_TYPE
ORDER BY Count(*)
""").show(truncate=False)

+-----------------------------------------+--------+
|NAME_PAYMENT_TYPE                        |count(1)|
+-----------------------------------------+--------+
|Cashless from the account of the employer|1085    |
|Non-cash from your account               |8193    |
|XNA                                      |627384  |
|Cash through the bank                    |1033552 |
+-----------------------------------------+--------+



In [28]:
spark.sql("""
SELECT
CODE_REJECT_REASON,
Count(*)
FROM
df_previous_application
GROUP BY CODE_REJECT_REASON
ORDER BY Count(*)
""").show()

+------------------+--------+
|CODE_REJECT_REASON|count(1)|
+------------------+--------+
|            SYSTEM|     717|
|             VERIF|    3535|
|               XNA|    5244|
|             SCOFR|   12811|
|            CLIENT|   26436|
|               SCO|   37467|
|             LIMIT|   55680|
|                HC|  175231|
|               XAP| 1353093|
+------------------+--------+



In [29]:
spark.sql("""
SELECT
NAME_TYPE_SUITE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_TYPE_SUITE
ORDER BY Count(*)
""").show()

+---------------+--------+
|NAME_TYPE_SUITE|count(1)|
+---------------+--------+
|Group of people|    2240|
|        Other_A|    9077|
|        Other_B|   17624|
|       Children|   31566|
|Spouse, partner|   67069|
|         Family|  213263|
|  Unaccompanied|  508970|
|           null|  820405|
+---------------+--------+



In [30]:
spark.sql("""
SELECT
NAME_CLIENT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_CLIENT_TYPE
ORDER BY Count(*)
""").show()

+----------------+--------+
|NAME_CLIENT_TYPE|count(1)|
+----------------+--------+
|             XNA|    1941|
|       Refreshed|  135649|
|             New|  301363|
|        Repeater| 1231261|
+----------------+--------+



In [31]:
spark.sql("""
SELECT
NAME_GOODS_CATEGORY,
Count(*)
FROM
df_previous_application
GROUP BY NAME_GOODS_CATEGORY
ORDER BY Count(*)
""").show(truncate=False)

+------------------------+--------+
|NAME_GOODS_CATEGORY     |count(1)|
+------------------------+--------+
|Animals                 |1       |
|House Construction      |1       |
|Insurance               |64      |
|Weapon                  |77      |
|Education               |107     |
|Additional Service      |128     |
|Fitness                 |209     |
|Direct Sales            |446     |
|Medicine                |1550    |
|Tourism                 |1659    |
|Office Appliances       |2333    |
|Other                   |2554    |
|Gardening               |2668    |
|Sport and Leisure       |2981    |
|Vehicles                |3370    |
|Medical Supplies        |3843    |
|Homewares               |5023    |
|Jewelry                 |6290    |
|Auto Accessories        |7381    |
|Clothing and Accessories|23554   |
+------------------------+--------+
only showing top 20 rows



In [32]:
spark.sql("""
SELECT
NAME_PORTFOLIO,
Count(*)
FROM
df_previous_application
GROUP BY NAME_PORTFOLIO
ORDER BY Count(*)
""").show()

+--------------+--------+
|NAME_PORTFOLIO|count(1)|
+--------------+--------+
|          Cars|     425|
|         Cards|  144985|
|           XNA|  372230|
|          Cash|  461563|
|           POS|  691011|
+--------------+--------+



In [33]:
spark.sql("""
SELECT
NAME_PRODUCT_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY NAME_PRODUCT_TYPE
ORDER BY Count(*)
""").show()

+-----------------+--------+
|NAME_PRODUCT_TYPE|count(1)|
+-----------------+--------+
|          walk-in|  150261|
|           x-sell|  456287|
|              XNA| 1063666|
+-----------------+--------+



In [34]:
spark.sql("""
SELECT
CHANNEL_TYPE,
Count(*)
FROM
df_previous_application
GROUP BY CHANNEL_TYPE
ORDER BY Count(*)
""").show(truncate=False)

+--------------------------+--------+
|CHANNEL_TYPE              |count(1)|
+--------------------------+--------+
|Car dealer                |452     |
|Channel of corporate sales|6150    |
|AP+ (Cash loan)           |57046   |
|Contact center            |71297   |
|Regional / Local          |108528  |
|Stone                     |212083  |
|Country-wide              |494690  |
|Credit and cash offices   |719968  |
+--------------------------+--------+



In [35]:
spark.sql("""
SELECT
NAME_SELLER_INDUSTRY,
Count(*)
FROM
df_previous_application
GROUP BY NAME_SELLER_INDUSTRY
ORDER BY Count(*)
""").show()

+--------------------+--------+
|NAME_SELLER_INDUSTRY|count(1)|
+--------------------+--------+
|             Tourism|     513|
|        MLM partners|    1215|
|             Jewelry|    2709|
|     Auto technology|    4990|
|            Industry|   19194|
|            Clothing|   23949|
|        Construction|   29781|
|           Furniture|   57849|
|        Connectivity|  276029|
|Consumer electronics|  398265|
|                 XNA|  855720|
+--------------------+--------+



In [36]:
spark.sql("""
SELECT
NAME_YIELD_GROUP,
Count(*)
FROM
df_previous_application
GROUP BY NAME_YIELD_GROUP
ORDER BY Count(*)
""").show()

+----------------+--------+
|NAME_YIELD_GROUP|count(1)|
+----------------+--------+
|      low_action|   92041|
|      low_normal|  322095|
|            high|  353331|
|          middle|  385532|
|             XNA|  517215|
+----------------+--------+



In [37]:
spark.sql("""
SELECT
PRODUCT_COMBINATION,
Count(*)
FROM
df_previous_application
GROUP BY PRODUCT_COMBINATION
ORDER BY Count(*)
""").show(truncate=False)

+------------------------------+--------+
|PRODUCT_COMBINATION           |count(1)|
+------------------------------+--------+
|null                          |346     |
|POS others without interest   |2555    |
|POS industry without interest |12602   |
|POS other with interest       |23879   |
|POS mobile without interest   |24082   |
|Cash Street: low              |33834   |
|Cash Street: middle           |34658   |
|Cash X-Sell: high             |59301   |
|Cash Street: high             |59639   |
|Card X-Sell                   |80582   |
|POS household without interest|82908   |
|POS industry with interest    |98833   |
|Card Street                   |112582  |
|Cash X-Sell: low              |130248  |
|Cash X-Sell: middle           |143883  |
|POS mobile with interest      |220670  |
|POS household with interest   |263622  |
|Cash                          |285990  |
+------------------------------+--------+



## Agregando apenas as variáveis categóricas e seus domínios


**OBS:** Para automatizar a criação de cada linha de agregação vamos desenvolver uma rotina que gera o texto de agregação. Após, basta susbstiruir na requesição SQL.

###1- Considerando a variável categórica NAME_CONTRACT_TYPE

In [38]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Revolving_loans', 'Consumer_loans', 'Cash_loans']

In [39]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Revolving_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Consumer_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Cash_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Revolving_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Consumer_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Cash_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END)

###2- Considerando a variável categórica WEEKDAY_APPR_PROCESS_START

In [40]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['WEEKDAY_APPR_PROCESS_START']

# Criando os domínios

dominios_cat = ['SUNDAY', 'SATURDAY', 'THURSDAY', 'FRIDAY','MONDAY','WEDNESDAY','TUESDAY']

In [41]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SUNDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SATURDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_THURSDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_FRIDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_MONDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_TUESDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SUNDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_A

###3 - Considerando a variável categórica FLAG_LAST_APPL_PER_CONTRACT

In [42]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['FLAG_LAST_APPL_PER_CONTRACT']

# Criando os domínios

dominios_cat = ['N', 'Y']

In [43]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,


###4 - Considerando a variável categórica NAME_CASH_LOAN_PURPOSE

In [44]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CASH_LOAN_PURPOSE']

# Criando os domínios

dominios_cat = ['Refusal_to_name_the_goal', 'Money_for_a_third_person', 'Hobby', 'Buying_a_garage', 'Gasification_water_supply', 'Business_development', 'Buying_a_holiday_home_land', 'Furniture', 'Car_repairs', 'Buying_a_home', 'Wedding_gift_holiday', 'Buying_a_new_car', 'Purchase_of_electronic_equipment', 'Journey', 'Education', 'Payments_on_other_loans', 'Medicine', 'Everyday_expenses', 'Building_a_house_or_an_annex', 'Buying_a_used_car']

In [45]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money_for_a_third_person' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Hobby,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_garage' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification_water_supply' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Gasification_water_supply,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business_development' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Business_development,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_holiday_home_land' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home_land,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NA

###5 - Considerando a variável categórica NAME_CONTRACT_STATUS

In [46]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_STATUS']

# Criando os domínios

dominios_cat = ['Unused_offer', 'Refused', 'Canceled', 'Approved']

In [47]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Refused,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Canceled,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Approved,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Refused,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Canceled,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Approved,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused

###6 - Considerando a variável categórica NAME_PAYMENT_TYPE

In [48]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PAYMENT_TYPE']

# Criando os domínios

dominios_cat = ['Cashless_from_the_account_of_the_employer', 'Non_cash_from_your_account', 'XNA', 'Cash_through_the_bank']

In [49]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_XNA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cash_through_the_bank,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_XNA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cas

###7 - Considerando a variável categórica CODE_REJECT_REASON

In [50]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CODE_REJECT_REASON']

# Criando os domínios

dominios_cat = ['SYSTEM', 'VERIF', 'XNA', 'SCOFR','CLIENT','SCO','LIMIT', 'HC','XAP']

In [51]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SYSTEM,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_VERIF,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XNA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCOFR,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_CLIENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCO,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_LIMIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_HC,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XAP,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SYSTEM,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS qtd_CODE

###8 - Considerando a variável categórica NAME_TYPE_SUITE

In [52]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_TYPE_SUITE']

# Criando os domínios

dominios_cat = ['Group_of_people', 'Other_A', 'Other_B', 'Children','Spouse_partner','Family','Unaccompanied', 'null']

In [53]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Group_of_people,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_A,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_B,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Children,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Spouse_partner,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Family,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Unaccompanied,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_null,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Group_of_people,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_A,
MIN(CASE WHEN NAME_TYPE_SUIT

###9 - Considerando a variável categórica NAME_CLIENT_TYPE

In [54]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CLIENT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Refreshed', 'New', 'Repeater']

In [55]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Repeater,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Repeater,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New

###10 - Considerando a variável categórica NAME_GOODS_CATEGORY

In [56]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_GOODS_CATEGORY']

# Criando os domínios

dominios_cat = ['House_Construction', 'Animals', 'Insurance', 'Education','Additional_Service','Fitness','Direct_Sales', 'Medicine', 'Tourism','Office_Appliances','Other','Gardening','Sport_and_Leisure','Vehicles','Medical_Supplies','Homewares','Jewelry','Auto_Accessories','Clothing_and_Accessories']

In [57]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_House_Construction,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Animals,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Insurance,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Education,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Additional_Service,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Fitness,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Direct_Sales,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medicine,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Tourism,
SUM(CASE WHEN NAME_GOODS_CATEGORY= '

###11 - Considerando a variável categórica NAME_PORTFOLIO

In [58]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PORTFOLIO']

# Criando os domínios

dominios_cat = ['Cars', 'Cards', 'XNA', 'Cash','POS']

In [59]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cards,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_XNA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cash,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_POS,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cards,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_XNA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cash,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_POS,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_C

###12 - Considerando a variável categórica NAME_PRODUCT_TYPE

In [60]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PRODUCT_TYPE']

# Criando os domínios

dominios_cat = ['walk_in', 'x_sell', 'XNA']

In [61]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NA

###13 - Considerando a variável categórica CHANNEL_TYPE

In [62]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CHANNEL_TYPE']

# Criando os domínios

dominios_cat = ['Car_dealer', 'Channel_of_corporate_sales', 'AP_Cash_loan','Contact_center','Regional_Local','Stone','Country_wide','Credit_and_cash_offices']

In [63]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Car_dealer,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Channel_of_corporate_sales,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_AP_Cash_loan,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Contact_center,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Regional_Local,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Stone,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Country_wide,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Credit_and_cash_offices,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Car_dealer,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Chan

###14 - Considerando a variável categórica NAME_SELLER_INDUSTRY

In [64]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_SELLER_INDUSTRY']

# Criando os domínios

dominios_cat = ['MLM_partners', 'Jewelry', 'Auto_technology','Industry','Clothing','Construction','Furniture','Connectivity','Consumer_electronics','XNA']

In [65]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_MLM_partners,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Jewelry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Auto_technology,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Industry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Clothing,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Construction,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Furniture,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Connectivity,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Consumer_electronics,
SU

###15 - Considerando a variável categórica NAME_YIELD_GROUP

In [66]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_YIELD_GROUP']

# Criando os domínios

dominios_cat = ['low_action', 'low_normal', 'high','middle','XNA']

In [67]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_action,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_normal,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_high,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_middle,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_XNA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_action,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_normal,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_high,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_middle,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_XNA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS q

###16 - Considerando a variável categórica PRODUCT_COMBINATION

In [68]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['PRODUCT_COMBINATION']

# Criando os domínios

dominios_cat = ['null', 'POS_others_without_interest', 'POS_industry_without_interest','POS_other_with_interest','POS_mobile_without_interest','Cash_Street_low','Cash_Street_middle','Cash_X_Sell_high','Cash_Street_high', 'Card_X_Sell','POS_household_without_interest','POS_industry_with_interest','Card_Street','Cash_X_Sell_low','Cash_X_Sell_middle','POS_mobile_with_interest','POS_household_with_interest','Cash']

In [69]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      #for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN 1 ELSE 0 END) AS qtd_" + cate + "_" + dominio +',')

SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_null,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_others_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_other_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_low,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_middle,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_

##Criação do dataframe df_previous_application_cat somente com agregações de variáveis categóricas

In [70]:
df_previous_application_cat = spark.sql("""
SELECT
SK_ID_CURR,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Revolving_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Consumer_loans,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Cash_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Revolving_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Consumer_loans,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Cash_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Revolving_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Consumer_loans,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Cash_loans,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_XNA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Revolving_loans,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Consumer_loans,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_TYPE_Cash_loans,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SUNDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SATURDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_THURSDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_FRIDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_MONDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_TUESDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SUNDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SATURDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_THURSDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_FRIDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_MONDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_TUESDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SUNDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SATURDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_THURSDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_FRIDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_MONDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_TUESDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SUNDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_SATURDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_THURSDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_FRIDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_MONDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN 1 ELSE 0 END) AS qtd_WEEKDAY_APPR_PROCESS_START_TUESDAY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_N,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN 1 ELSE 0 END) AS qtd_FLAG_LAST_APPL_PER_CONTRACT_Y,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Refused,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Canceled,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Approved,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Refused,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Canceled,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Approved,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Refused,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Canceled,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Approved,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Unused_offer,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Refused,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Canceled,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN 1 ELSE 0 END) AS qtd_NAME_CONTRACT_STATUS_Approved,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_XNA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cash_through_the_bank,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_XNA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cash_through_the_bank,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_XNA,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cash_through_the_bank,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Non_cash_from_your_account,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_XNA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN 1 ELSE 0 END) AS qtd_NAME_PAYMENT_TYPE_Cash_through_the_bank,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SYSTEM,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_VERIF,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XNA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCOFR,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_CLIENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCO,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_LIMIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_HC,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XAP,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SYSTEM,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_VERIF,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XNA,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCOFR,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_CLIENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCO,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_LIMIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_HC,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XAP,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SYSTEM,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_VERIF,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XNA,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCOFR,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_CLIENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCO,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_LIMIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_HC,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XAP,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SYSTEM,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_VERIF,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XNA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCOFR,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_CLIENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_SCO,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_LIMIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_HC,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN 1 ELSE 0 END) AS qtd_CODE_REJECT_REASON_XAP,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Group_of_people,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_A,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_B,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Children,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Spouse_partner,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Family,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Unaccompanied,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_null,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Group_of_people,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_A,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_B,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Children,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Spouse_partner,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Family,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Unaccompanied,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_null,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Group_of_people,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_A,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_B,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Children,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Spouse_partner,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Family,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Unaccompanied,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_null,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Group_of_people,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_A,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Other_B,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Children,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Spouse_partner,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Family,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_Unaccompanied,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN 1 ELSE 0 END) AS qtd_NAME_TYPE_SUITE_null,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Repeater,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New,
MIN(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Repeater,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New,
MAX(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Repeater,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_XNA,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Refreshed' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Refreshed,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'New' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_New,
AVG(CASE WHEN NAME_CLIENT_TYPE= 'Repeater' THEN 1 ELSE 0 END) AS qtd_NAME_CLIENT_TYPE_Repeater,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_House_Construction,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Animals,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Insurance,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Education,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Additional_Service,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Fitness,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Direct_Sales,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medicine,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Tourism,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Office_Appliances,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Other,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Gardening,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Sport_and_Leisure,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Vehicles,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medical_Supplies,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Homewares,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Jewelry,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Auto_Accessories,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_House_Construction,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Animals,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Insurance,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Education,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Additional_Service,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Fitness,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Direct_Sales,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medicine,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Tourism,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Office_Appliances,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Other,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Gardening,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Sport_and_Leisure,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Vehicles,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medical_Supplies,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Homewares,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Jewelry,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Auto_Accessories,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_House_Construction,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Animals,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Insurance,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Education,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Additional_Service,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Fitness,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Direct_Sales,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medicine,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Tourism,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Office_Appliances,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Other,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Gardening,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Sport_and_Leisure,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Vehicles,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medical_Supplies,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Homewares,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Jewelry,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Auto_Accessories,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_House_Construction,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Animals,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Insurance,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Education,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Additional_Service,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Fitness,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Direct_Sales,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medicine,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Tourism,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Office_Appliances,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Other,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Gardening,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Sport_and_Leisure,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Vehicles,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Medical_Supplies,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Homewares,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Jewelry,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Auto_Accessories,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN 1 ELSE 0 END) AS qtd_NAME_GOODS_CATEGORY_Clothing_and_Accessories,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cards,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_XNA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cash,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_POS,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cards,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_XNA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cash,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_POS,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cards,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_XNA,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cash,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_POS,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cars,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cards,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_XNA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_Cash,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN 1 ELSE 0 END) AS qtd_NAME_PORTFOLIO_POS,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_walk_in,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_x_sell,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_PRODUCT_TYPE_XNA,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Car_dealer,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Channel_of_corporate_sales,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_AP_Cash_loan,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Contact_center,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Regional_Local,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Stone,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Country_wide,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Credit_and_cash_offices,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Car_dealer,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Channel_of_corporate_sales,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_AP_Cash_loan,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Contact_center,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Regional_Local,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Stone,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Country_wide,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Credit_and_cash_offices,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Car_dealer,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Channel_of_corporate_sales,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_AP_Cash_loan,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Contact_center,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Regional_Local,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Stone,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Country_wide,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Credit_and_cash_offices,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Car_dealer,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Channel_of_corporate_sales,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_AP_Cash_loan,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Contact_center,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Regional_Local,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Stone,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Country_wide,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN 1 ELSE 0 END) AS qtd_CHANNEL_TYPE_Credit_and_cash_offices,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_MLM_partners,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Jewelry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Auto_technology,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Industry,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Clothing,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Construction,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Furniture,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Connectivity,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Consumer_electronics,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_XNA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_MLM_partners,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Jewelry,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Auto_technology,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Industry,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Clothing,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Construction,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Furniture,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Connectivity,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Consumer_electronics,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_XNA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_MLM_partners,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Jewelry,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Auto_technology,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Industry,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Clothing,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Construction,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Furniture,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Connectivity,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Consumer_electronics,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_XNA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_MLM_partners,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Jewelry,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Auto_technology,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Industry,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Clothing,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Construction,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Furniture,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Connectivity,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_Consumer_electronics,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_SELLER_INDUSTRY_XNA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_action,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_normal,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_high,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_middle,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_XNA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_action,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_normal,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_high,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_middle,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_XNA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_action,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_normal,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_high,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_middle,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_XNA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_action,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_low_normal,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_high,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_middle,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN 1 ELSE 0 END) AS qtd_NAME_YIELD_GROUP_XNA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_null,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_others_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_other_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_low,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_middle,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_high,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_high,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_X_Sell,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_without_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_Street,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_low,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_middle,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_with_interest,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_null,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_others_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_other_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_low,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_middle,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_high,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_high,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_X_Sell,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_without_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_Street,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_low,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_middle,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_with_interest,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_null,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_others_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_other_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_low,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_middle,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_high,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_high,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_X_Sell,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_without_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_Street,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_low,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_middle,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_with_interest,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_null,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_others_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_other_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_low,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_middle,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_high,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_Street_high,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_X_Sell,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_without_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_industry_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Card_Street,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_low,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash_X_Sell_middle,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_mobile_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_POS_household_with_interest,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN 1 ELSE 0 END) AS qtd_PRODUCT_COMBINATION_Cash,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money_for_a_third_person' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Hobby,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_garage' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification_water_supply' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Gasification_water_supply,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business_development' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Business_development,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_holiday_home_land' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home_land,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Furniture,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car_repairs' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Car_repairs,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_home' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding_gift_holiday' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Wedding_gift_holiday,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_new_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase_of_electronic_equipment' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Journey,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Education,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments_on_other_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Medicine,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday_expenses' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building_a_house_or_an_annex' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_used_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money_for_a_third_person' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Hobby,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_garage' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification_water_supply' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Gasification_water_supply,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business_development' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Business_development,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_holiday_home_land' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home_land,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Furniture,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car_repairs' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Car_repairs,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_home' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding_gift_holiday' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Wedding_gift_holiday,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_new_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase_of_electronic_equipment' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Journey,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Education,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments_on_other_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Medicine,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday_expenses' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building_a_house_or_an_annex' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
MIN(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_used_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money_for_a_third_person' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Hobby,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_garage' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification_water_supply' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Gasification_water_supply,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business_development' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Business_development,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_holiday_home_land' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home_land,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Furniture,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car_repairs' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Car_repairs,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_home' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding_gift_holiday' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Wedding_gift_holiday,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_new_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase_of_electronic_equipment' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Journey,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Education,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments_on_other_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Medicine,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday_expenses' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building_a_house_or_an_annex' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
MAX(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_used_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Money_for_a_third_person' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Money_for_a_third_person,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Hobby' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Hobby,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_garage' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_garage,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Gasification_water_supply' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Gasification_water_supply,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Business_development' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Business_development,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_holiday_home_land' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_holiday_home_land,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Furniture' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Furniture,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Car_repairs' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Car_repairs,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_home' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_home,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Wedding_gift_holiday' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Wedding_gift_holiday,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_new_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_new_car,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Purchase_of_electronic_equipment' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Purchase_of_electronic_equipment,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Journey' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Journey,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Education' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Education,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Payments_on_other_loans' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Payments_on_other_loans,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Medicine' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Medicine,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Everyday_expenses' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Everyday_expenses,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Building_a_house_or_an_annex' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Building_a_house_or_an_annex,
AVG(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Buying_a_used_car' THEN 1 ELSE 0 END) AS qtd_NAME_CASH_LOAN_PURPOSE_Buying_a_used_car
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")

In [71]:
df_previous_application_cat.show()

+----------+--------------------------+--------------------------------------+-------------------------------------+---------------------------------+--------------------------+--------------------------------------+-------------------------------------+---------------------------------+--------------------------+--------------------------------------+-------------------------------------+---------------------------------+--------------------------+--------------------------------------+-------------------------------------+---------------------------------+-------------------------------------+---------------------------------------+---------------------------------------+-------------------------------------+-------------------------------------+----------------------------------------+--------------------------------------+-------------------------------------+---------------------------------------+---------------------------------------+-------------------------------------+------

In [72]:
df_previous_application_cat.createOrReplaceTempView("df_previous_application_cat")

##Transformando as variáveis numéricas em agregações

Após criar o dataframe df_previous_application_cat somente com agregações de variáveis categóricas, podemos criar outros dataframes com agregações entre as variáveis categóricas e numéricas. As variáveis numéricas da tabela df_previous_application são:

OBS: Para automatizar a criação de cada linha de agregação vamos desenvolver uma rotina que gera o texto de agregação. Após, basta susbstiruir na requesição SQL.


- AMT_ANNUITY: double
- AMT_APPLICATION: double
- AMT_CREDIT: double
- AMT_DOWN_PAYMENT: double
- AMT_GOODS_PRICE: double
- HOUR_APPR_PROCESS_START: integer
- NFLAG_LAST_APPL_IN_DAY: integer
- RATE_DOWN_PAYMENT: double
- RATE_INTEREST_PRIMARY: double
- RATE_INTEREST_PRIVILEGED: double
- DAYS_DECISION: integer
- SELLERPLACE_AREA: integer
- CNT_PAYMENT: double
- DAYS_FIRST_DRAWING: double
- DAYS_FIRST_DUE: double
- DAYS_LAST_DUE_1ST_VERSION: double
- DAYS_LAST_DUE: double
- DAYS_TERMINATION: double
- NFLAG_INSURED_ON_APPROVAL: double


In [73]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável numérica e de data

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [74]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for var_n in var_num:
        print(ag + "(" + var_n + ") AS " + ag + "_" + var_n + ',')

SUM(AMT_ANNUITY) AS SUM_AMT_ANNUITY,
SUM(AMT_APPLICATION) AS SUM_AMT_APPLICATION,
SUM(AMT_CREDIT) AS SUM_AMT_CREDIT,
SUM(AMT_DOWN_PAYMENT) AS SUM_AMT_DOWN_PAYMENT,
SUM(AMT_GOODS_PRICE) AS SUM_AMT_GOODS_PRICE,
SUM(HOUR_APPR_PROCESS_START) AS SUM_HOUR_APPR_PROCESS_START,
SUM(NFLAG_LAST_APPL_IN_DAY) AS SUM_NFLAG_LAST_APPL_IN_DAY,
SUM(RATE_DOWN_PAYMENT) AS SUM_RATE_DOWN_PAYMENT,
SUM(RATE_INTEREST_PRIMARY) AS SUM_RATE_INTEREST_PRIMARY,
SUM(RATE_INTEREST_PRIVILEGED) AS SUM_RATE_INTEREST_PRIVILEGED,
SUM(DAYS_DECISION) AS SUM_DAYS_DECISION,
SUM(SELLERPLACE_AREA) AS SUM_SELLERPLACE_AREA,
SUM(CNT_PAYMENT) AS SUM_CNT_PAYMENT,
SUM(DAYS_FIRST_DRAWING) AS SUM_DAYS_FIRST_DRAWING,
SUM(DAYS_FIRST_DUE) AS SUM_DAYS_FIRST_DUE,
SUM(DAYS_LAST_DUE_1ST_VERSION) AS SUM_DAYS_LAST_DUE_1ST_VERSION,
SUM(DAYS_LAST_DUE) AS SUM_DAYS_LAST_DUE,
SUM(DAYS_TERMINATION) AS SUM_DAYS_TERMINATION,
SUM(NFLAG_INSURED_ON_APPROVAL) AS SUM_NFLAG_INSURED_ON_APPROVAL,
MIN(AMT_ANNUITY) AS MIN_AMT_ANNUITY,
MIN(AMT_APPLICATION) AS MIN_

###Criação do dataframe df_previous_application_num somente com agregações de variáveis numericas

In [75]:
df_previous_application_num = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(AMT_ANNUITY) AS SUM_AMT_ANNUITY,
SUM(AMT_APPLICATION) AS SUM_AMT_APPLICATION,
SUM(AMT_CREDIT) AS SUM_AMT_CREDIT,
SUM(AMT_DOWN_PAYMENT) AS SUM_AMT_DOWN_PAYMENT,
SUM(AMT_GOODS_PRICE) AS SUM_AMT_GOODS_PRICE,
SUM(HOUR_APPR_PROCESS_START) AS SUM_HOUR_APPR_PROCESS_START,
SUM(NFLAG_LAST_APPL_IN_DAY) AS SUM_NFLAG_LAST_APPL_IN_DAY,
SUM(RATE_DOWN_PAYMENT) AS SUM_RATE_DOWN_PAYMENT,
SUM(RATE_INTEREST_PRIMARY) AS SUM_RATE_INTEREST_PRIMARY,
SUM(RATE_INTEREST_PRIVILEGED) AS SUM_RATE_INTEREST_PRIVILEGED,
SUM(DAYS_DECISION) AS SUM_DAYS_DECISION,
SUM(SELLERPLACE_AREA) AS SUM_SELLERPLACE_AREA,
SUM(CNT_PAYMENT) AS SUM_CNT_PAYMENT,
SUM(DAYS_FIRST_DRAWING) AS SUM_DAYS_FIRST_DRAWING,
SUM(DAYS_FIRST_DUE) AS SUM_DAYS_FIRST_DUE,
SUM(DAYS_LAST_DUE_1ST_VERSION) AS SUM_DAYS_LAST_DUE_1ST_VERSION,
SUM(DAYS_LAST_DUE) AS SUM_DAYS_LAST_DUE,
SUM(DAYS_TERMINATION) AS SUM_DAYS_TERMINATION,
SUM(NFLAG_INSURED_ON_APPROVAL) AS SUM_NFLAG_INSURED_ON_APPROVAL,
MIN(AMT_ANNUITY) AS MIN_AMT_ANNUITY,
MIN(AMT_APPLICATION) AS MIN_AMT_APPLICATION,
MIN(AMT_CREDIT) AS MIN_AMT_CREDIT,
MIN(AMT_DOWN_PAYMENT) AS MIN_AMT_DOWN_PAYMENT,
MIN(AMT_GOODS_PRICE) AS MIN_AMT_GOODS_PRICE,
MIN(HOUR_APPR_PROCESS_START) AS MIN_HOUR_APPR_PROCESS_START,
MIN(NFLAG_LAST_APPL_IN_DAY) AS MIN_NFLAG_LAST_APPL_IN_DAY,
MIN(RATE_DOWN_PAYMENT) AS MIN_RATE_DOWN_PAYMENT,
MIN(RATE_INTEREST_PRIMARY) AS MIN_RATE_INTEREST_PRIMARY,
MIN(RATE_INTEREST_PRIVILEGED) AS MIN_RATE_INTEREST_PRIVILEGED,
MIN(DAYS_DECISION) AS MIN_DAYS_DECISION,
MIN(SELLERPLACE_AREA) AS MIN_SELLERPLACE_AREA,
MIN(CNT_PAYMENT) AS MIN_CNT_PAYMENT,
MIN(DAYS_FIRST_DRAWING) AS MIN_DAYS_FIRST_DRAWING,
MIN(DAYS_FIRST_DUE) AS MIN_DAYS_FIRST_DUE,
MIN(DAYS_LAST_DUE_1ST_VERSION) AS MIN_DAYS_LAST_DUE_1ST_VERSION,
MIN(DAYS_LAST_DUE) AS MIN_DAYS_LAST_DUE,
MIN(DAYS_TERMINATION) AS MIN_DAYS_TERMINATION,
MIN(NFLAG_INSURED_ON_APPROVAL) AS MIN_NFLAG_INSURED_ON_APPROVAL,
MAX(AMT_ANNUITY) AS MAX_AMT_ANNUITY,
MAX(AMT_APPLICATION) AS MAX_AMT_APPLICATION,
MAX(AMT_CREDIT) AS MAX_AMT_CREDIT,
MAX(AMT_DOWN_PAYMENT) AS MAX_AMT_DOWN_PAYMENT,
MAX(AMT_GOODS_PRICE) AS MAX_AMT_GOODS_PRICE,
MAX(HOUR_APPR_PROCESS_START) AS MAX_HOUR_APPR_PROCESS_START,
MAX(NFLAG_LAST_APPL_IN_DAY) AS MAX_NFLAG_LAST_APPL_IN_DAY,
MAX(RATE_DOWN_PAYMENT) AS MAX_RATE_DOWN_PAYMENT,
MAX(RATE_INTEREST_PRIMARY) AS MAX_RATE_INTEREST_PRIMARY,
MAX(RATE_INTEREST_PRIVILEGED) AS MAX_RATE_INTEREST_PRIVILEGED,
MAX(DAYS_DECISION) AS MAX_DAYS_DECISION,
MAX(SELLERPLACE_AREA) AS MAX_SELLERPLACE_AREA,
MAX(CNT_PAYMENT) AS MAX_CNT_PAYMENT,
MAX(DAYS_FIRST_DRAWING) AS MAX_DAYS_FIRST_DRAWING,
MAX(DAYS_FIRST_DUE) AS MAX_DAYS_FIRST_DUE,
MAX(DAYS_LAST_DUE_1ST_VERSION) AS MAX_DAYS_LAST_DUE_1ST_VERSION,
MAX(DAYS_LAST_DUE) AS MAX_DAYS_LAST_DUE,
MAX(DAYS_TERMINATION) AS MAX_DAYS_TERMINATION,
MAX(NFLAG_INSURED_ON_APPROVAL) AS MAX_NFLAG_INSURED_ON_APPROVAL,
AVG(AMT_ANNUITY) AS AVG_AMT_ANNUITY,
AVG(AMT_APPLICATION) AS AVG_AMT_APPLICATION,
AVG(AMT_CREDIT) AS AVG_AMT_CREDIT,
AVG(AMT_DOWN_PAYMENT) AS AVG_AMT_DOWN_PAYMENT,
AVG(AMT_GOODS_PRICE) AS AVG_AMT_GOODS_PRICE,
AVG(HOUR_APPR_PROCESS_START) AS AVG_HOUR_APPR_PROCESS_START,
AVG(NFLAG_LAST_APPL_IN_DAY) AS AVG_NFLAG_LAST_APPL_IN_DAY,
AVG(RATE_DOWN_PAYMENT) AS AVG_RATE_DOWN_PAYMENT,
AVG(RATE_INTEREST_PRIMARY) AS AVG_RATE_INTEREST_PRIMARY,
AVG(RATE_INTEREST_PRIVILEGED) AS AVG_RATE_INTEREST_PRIVILEGED,
AVG(DAYS_DECISION) AS AVG_DAYS_DECISION,
AVG(SELLERPLACE_AREA) AS AVG_SELLERPLACE_AREA,
AVG(CNT_PAYMENT) AS AVG_CNT_PAYMENT,
AVG(DAYS_FIRST_DRAWING) AS AVG_DAYS_FIRST_DRAWING,
AVG(DAYS_FIRST_DUE) AS AVG_DAYS_FIRST_DUE,
AVG(DAYS_LAST_DUE_1ST_VERSION) AS AVG_DAYS_LAST_DUE_1ST_VERSION,
AVG(DAYS_LAST_DUE) AS AVG_DAYS_LAST_DUE,
AVG(DAYS_TERMINATION) AS AVG_DAYS_TERMINATION,
AVG(NFLAG_INSURED_ON_APPROVAL) AS AVG_NFLAG_INSURED_ON_APPROVAL
FROM
df_previous_application
GROUP BY
  SK_ID_CURR
""")

In [76]:
df_previous_application_num.show()

+----------+---------------+-------------------+--------------+--------------------+-------------------+---------------------------+--------------------------+---------------------+-------------------------+----------------------------+-----------------+--------------------+---------------+----------------------+------------------+-----------------------------+-----------------+--------------------+-----------------------------+---------------+-------------------+--------------+--------------------+-------------------+---------------------------+--------------------------+---------------------+-------------------------+----------------------------+-----------------+--------------------+---------------+----------------------+------------------+-----------------------------+-----------------+--------------------+-----------------------------+---------------+-------------------+--------------+--------------------+-------------------+---------------------------+--------------------------+-

In [77]:
df_previous_application_num.createOrReplaceTempView("df_previous_application_num")

##Criando as variáveis numéricas e atrelando a diferentes status das variáveis categóricas

###1- Considerando a variável categórica NAME_CONTRACT_TYPE

In [78]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Revolving_loans', 'Consumer_loans', 'Cash_loans']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [79]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CON

###2- Considerando a variável categórica WEEKDAY_APPR_PROCESS_START

In [80]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['WEEKDAY_APPR_PROCESS_START']

# Criando os domínios

dominios_cat = ['SUNDAY', 'SATURDAY', 'THURSDAY', 'FRIDAY','MONDAY','WEDNESDAY','TUESDAY']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [81]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_

###3 - Considerando a variável categórica FLAG_LAST_APPL_PER_CONTRACT

In [82]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['FLAG_LAST_APPL_PER_CONTRACT']

# Criando os domínios

dominios_cat = ['N', 'Y']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [83]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' T

###4 - Considerando a variável categórica NAME_CASH_LOAN_PURPOSE

In [84]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CASH_LOAN_PURPOSE']

# Criando os domínios

dominios_cat = ['Refusal_to_name_the_goal', 'Money_for_a_third_person', 'Hobby', 'Buying_a_garage', 'Gasification_water_supply', 'Business_development', 'Buying_a_holiday_home_land', 'Furniture', 'Car_repairs', 'Buying_a_home', 'Wedding_gift_holiday', 'Buying_a_new_car', 'Purchase_of_electronic_equipment', 'Journey', 'Education', 'Payments_on_other_loans', 'Medicine', 'Everyday_expenses', 'Building_a_house_or_an_annex', 'Buying_a_used_car']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [85]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_ANNUITY,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_APPLICATION,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_CREDIT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CASH_LOAN_PURPOSE= 'Refusal_to_name_the_goal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CASH_LOAN_PURPOSE_Refusal_to_name_the_goal_HOUR_

###5 - Considerando a variável categórica NAME_CONTRACT_STATUS

In [86]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_STATUS']

# Criando os domínios

dominios_cat = ['Unused_offer', 'Refused', 'Canceled', 'Approved']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [87]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_

###6 - Considerando a variável categórica NAME_PAYMENT_TYPE

In [88]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PAYMENT_TYPE']

# Criando os domínios

dominios_cat = ['Cashless_from_the_account_of_the_employer', 'Non_cash_from_your_account', 'XNA', 'Cash_through_the_bank']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [89]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_fro

###7 - Considerando a variável categórica CODE_REJECT_REASON

In [90]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CODE_REJECT_REASON']

# Criando os domínios

dominios_cat = ['SYSTEM', 'VERIF', 'XNA', 'SCOFR','CLIENT','SCO','LIMIT', 'HC','XAP']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [91]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_

###8 - Considerando a variável categórica NAME_TYPE_SUITE

In [92]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_TYPE_SUITE']

# Criando os domínios

dominios_cat = ['Group_of_people', 'Other_A', 'Other_B', 'Children','Spouse_partner','Family','Unaccompanied', 'null']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [93]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_T

###9 - Considerando a variável categórica NAME_CLIENT_TYPE

In [94]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CLIENT_TYPE']

# Criando os domínios

dominios_cat = ['XNA', 'Refreshed', 'New', 'Repeater']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [95]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CLIENT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CLIENT_TYPE= 'XNA' THEN RATE_INTERE

###10 - Considerando a variável categórica NAME_GOODS_CATEGORY

In [96]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_GOODS_CATEGORY']

# Criando os domínios

dominios_cat = ['House_Construction', 'Animals', 'Insurance', 'Education','Additional_Service','Fitness','Direct_Sales', 'Medicine', 'Tourism','Office_Appliances','Other','Gardening','Sport_and_Leisure','Vehicles','Medical_Supplies','Homewares','Jewelry','Auto_Accessories','Clothing_and_Accessories']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [97]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE

###11 - Considerando a variável categórica NAME_PORTFOLIO

In [98]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PORTFOLIO']

# Criando os domínios

dominios_cat = ['Cars', 'Cards', 'XNA', 'Cash','POS']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [99]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0

###12 - Considerando a variável categórica NAME_PRODUCT_TYPE

In [100]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_PRODUCT_TYPE']

# Criando os domínios

dominios_cat = ['walk_in', 'x_sell', 'XNA']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [101]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_w

###13 - Considerando a variável categórica CHANNEL_TYPE

In [102]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CHANNEL_TYPE']

# Criando os domínios

dominios_cat = ['Car_dealer', 'Channel_of_corporate_sales', 'AP_Cash_loan','Contact_center','Regional_Local','Stone','Country_wide','Credit_and_cash_offices']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [103]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
SUM(CA

###14 - Considerando a variável categórica NAME_SELLER_INDUSTRY

In [104]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_SELLER_INDUSTRY']

# Criando os domínios

dominios_cat = ['MLM_partners', 'Jewelry', 'Auto_technology','Industry','Clothing','Construction','Furniture','Connectivity','Consumer_electronics','XNA']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [105]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_

###15 - Considerando a variável categórica NAME_YIELD_GROUP

In [106]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_YIELD_GROUP']

# Criando os domínios

dominios_cat = ['low_action', 'low_normal', 'high','middle','XNA']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [107]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 E

###16 - Considerando a variável categórica PRODUCT_COMBINATION

In [108]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['PRODUCT_COMBINATION']

# Criando os domínios

dominios_cat = ['null', 'POS_others_without_interest', 'POS_industry_without_interest','POS_other_with_interest','POS_mobile_without_interest','Cash_Street_low','Cash_Street_middle','Cash_X_Sell_high','Cash_Street_high', 'Card_X_Sell','POS_household_without_interest','POS_industry_with_interest','Card_Street','Cash_X_Sell_low','Cash_X_Sell_middle','POS_mobile_with_interest','POS_household_with_interest','Cash']

# Criando variável numérica

var_num = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE','HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']

In [109]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_RATE_DOWN

##Criação da tabela df_previous_application_contract_num_and_cat

In [114]:
df_previous_application_contract_num_and_cat = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Revolving_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Revolving_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Consumer_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Consumer_loans_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_TYPE= 'Cash_loans' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_TYPE_Cash_loans_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_ANNUITY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_ANNUITY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_APPLICATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_APPLICATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_CREDIT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_CREDIT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_GOODS_PRICE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_DOWN_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_DECISION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_DECISION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_SELLERPLACE_AREA,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN CNT_PAYMENT ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_CNT_PAYMENT,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DRAWING,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_TERMINATION,
SUM(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_ANNUITY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_ANNUITY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_APPLICATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_APPLICATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_CREDIT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_CREDIT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_GOODS_PRICE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_DOWN_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_DECISION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_DECISION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_SELLERPLACE_AREA,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN CNT_PAYMENT ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_CNT_PAYMENT,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DRAWING,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_TERMINATION,
MIN(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_ANNUITY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_ANNUITY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_APPLICATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_APPLICATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_CREDIT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_CREDIT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_GOODS_PRICE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_DOWN_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_DECISION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_DECISION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_SELLERPLACE_AREA,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN CNT_PAYMENT ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_CNT_PAYMENT,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DRAWING,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_TERMINATION,
MAX(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SUNDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SUNDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'SATURDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_SATURDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'THURSDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_THURSDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'FRIDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_FRIDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'MONDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_MONDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'WEDNESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_ANNUITY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_ANNUITY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_APPLICATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_APPLICATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_CREDIT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_CREDIT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_AMT_GOODS_PRICE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_DOWN_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_DECISION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_DECISION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_SELLERPLACE_AREA,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN CNT_PAYMENT ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_CNT_PAYMENT,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DRAWING,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_FIRST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_LAST_DUE,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_DAYS_TERMINATION,
AVG(CASE WHEN WEEKDAY_APPR_PROCESS_START= 'TUESDAY' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_WEEKDAY_APPR_PROCESS_START_TUESDAY_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_DECISION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_DECISION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_SELLERPLACE_AREA,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN CNT_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_CNT_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DRAWING,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_TERMINATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_ANNUITY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_ANNUITY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_APPLICATION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_APPLICATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_CREDIT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_CREDIT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_GOODS_PRICE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_DOWN_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_DECISION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_DECISION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_SELLERPLACE_AREA,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN CNT_PAYMENT ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_CNT_PAYMENT,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DRAWING,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_TERMINATION,
SUM(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_DOWN_PAYMENT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_DECISION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_DECISION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_SELLERPLACE_AREA,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN CNT_PAYMENT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_CNT_PAYMENT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DRAWING,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DUE,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_TERMINATION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_ANNUITY ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_ANNUITY,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_APPLICATION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_APPLICATION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_CREDIT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_CREDIT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_DOWN_PAYMENT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_GOODS_PRICE,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_DOWN_PAYMENT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_DECISION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_DECISION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_SELLERPLACE_AREA,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN CNT_PAYMENT ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_CNT_PAYMENT,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DRAWING,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DUE,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_TERMINATION,
MIN(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_DOWN_PAYMENT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_DECISION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_DECISION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_SELLERPLACE_AREA,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN CNT_PAYMENT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_CNT_PAYMENT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DRAWING,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DUE,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_TERMINATION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_ANNUITY ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_ANNUITY,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_APPLICATION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_APPLICATION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_CREDIT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_CREDIT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_DOWN_PAYMENT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_GOODS_PRICE,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_DOWN_PAYMENT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_DECISION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_DECISION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_SELLERPLACE_AREA,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN CNT_PAYMENT ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_CNT_PAYMENT,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DRAWING,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DUE,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_TERMINATION,
MAX(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_ANNUITY ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_ANNUITY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_APPLICATION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_APPLICATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_CREDIT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_CREDIT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_AMT_GOODS_PRICE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_DECISION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_DECISION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_SELLERPLACE_AREA,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN CNT_PAYMENT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_CNT_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DRAWING,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_FIRST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_LAST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_DAYS_TERMINATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'N' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_N_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_ANNUITY ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_ANNUITY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_APPLICATION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_APPLICATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_CREDIT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_CREDIT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_AMT_GOODS_PRICE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_DOWN_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_DECISION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_DECISION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_SELLERPLACE_AREA,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN CNT_PAYMENT ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_CNT_PAYMENT,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DRAWING,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_FIRST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_LAST_DUE,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_DAYS_TERMINATION,
AVG(CASE WHEN FLAG_LAST_APPL_PER_CONTRACT= 'Y' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_FLAG_LAST_APPL_PER_CONTRACT_Y_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Canceled_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_ANNUITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_APPLICATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_DAYS_DECISION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_CNT_PAYMENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_DAYS_TERMINATION,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Canceled_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_ANNUITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_APPLICATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_DAYS_DECISION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_CNT_PAYMENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_DAYS_TERMINATION,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Canceled_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_ANNUITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_APPLICATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_DAYS_DECISION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_CNT_PAYMENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_DAYS_TERMINATION,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Unused_offer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Unused_offer_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Canceled' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Canceled_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_ANNUITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_APPLICATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_DAYS_DECISION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_CNT_PAYMENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_DAYS_TERMINATION,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_ANNUITY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_APPLICATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_CREDIT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_DECISION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_CNT_PAYMENT,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_DECISION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_CNT_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_ANNUITY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_APPLICATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_CREDIT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_DECISION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_CNT_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_AMT_ANNUITY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_AMT_APPLICATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_AMT_CREDIT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_DAYS_DECISION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_CNT_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_ANNUITY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_APPLICATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_CREDIT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_DECISION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_CNT_PAYMENT,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_DECISION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_CNT_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_ANNUITY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_APPLICATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_CREDIT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_DECISION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_CNT_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_AMT_ANNUITY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_AMT_APPLICATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_AMT_CREDIT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_DAYS_DECISION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_CNT_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_ANNUITY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_APPLICATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_CREDIT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_DECISION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_CNT_PAYMENT,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cashless_from_the_account_of_the_employer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cashless_from_the_account_of_the_employer_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Non_cash_from_your_account' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Non_cash_from_your_account_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_ANNUITY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_APPLICATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_CREDIT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_DECISION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_CNT_PAYMENT,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PAYMENT_TYPE= 'Cash_through_the_bank' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PAYMENT_TYPE_Cash_through_the_bank_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_SYSTEM_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_VERIF_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCOFR_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_CLIENT_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_SCO_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_LIMIT_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_HC_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_AMT_ANNUITY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_AMT_APPLICATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_CREDIT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_AMT_CREDIT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_AMT_GOODS_PRICE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_DECISION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_DAYS_DECISION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_SELLERPLACE_AREA,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_CNT_PAYMENT,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_DAYS_FIRST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_DAYS_TERMINATION,
SUM(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CODE_REJECT_REASON_XAP_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_SYSTEM_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_VERIF_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCOFR_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_CLIENT_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_SCO_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_LIMIT_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_HC_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_AMT_ANNUITY,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_AMT_APPLICATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_CREDIT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_AMT_CREDIT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_AMT_GOODS_PRICE,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_DECISION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_DAYS_DECISION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_SELLERPLACE_AREA,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_CNT_PAYMENT,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_DAYS_FIRST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_DAYS_TERMINATION,
MIN(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CODE_REJECT_REASON_XAP_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_SYSTEM_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_VERIF_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCOFR_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_CLIENT_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_SCO_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_LIMIT_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_HC_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_AMT_ANNUITY,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_AMT_APPLICATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_CREDIT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_AMT_CREDIT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_AMT_GOODS_PRICE,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_DECISION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_DAYS_DECISION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_SELLERPLACE_AREA,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_CNT_PAYMENT,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_DAYS_FIRST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_DAYS_TERMINATION,
MAX(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CODE_REJECT_REASON_XAP_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SYSTEM' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_SYSTEM_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'VERIF' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_VERIF_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCOFR' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCOFR_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'CLIENT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_CLIENT_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'SCO' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_SCO_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'LIMIT' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_LIMIT_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'HC' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_HC_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_AMT_ANNUITY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_AMT_APPLICATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_CREDIT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_AMT_CREDIT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_AMT_GOODS_PRICE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_DECISION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_DAYS_DECISION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_SELLERPLACE_AREA,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_CNT_PAYMENT,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_DAYS_FIRST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_DAYS_LAST_DUE,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_DAYS_TERMINATION,
AVG(CASE WHEN CODE_REJECT_REASON= 'XAP' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CODE_REJECT_REASON_XAP_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Group_of_people_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_A_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Other_B_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Children_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Spouse_partner_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Family_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_Unaccompanied_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_AMT_ANNUITY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_AMT_APPLICATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_AMT_CREDIT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_DAYS_DECISION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_CNT_PAYMENT,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_DAYS_TERMINATION,
SUM(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_TYPE_SUITE_null_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Group_of_people_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_A_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Other_B_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Children_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Spouse_partner_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Family_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_Unaccompanied_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_AMT_ANNUITY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_AMT_APPLICATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_AMT_CREDIT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_DAYS_DECISION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_CNT_PAYMENT,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_DAYS_TERMINATION,
MIN(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_TYPE_SUITE_null_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Group_of_people_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_A_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Other_B_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Children_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Spouse_partner_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Family_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_Unaccompanied_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_AMT_ANNUITY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_AMT_APPLICATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_AMT_CREDIT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_DAYS_DECISION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_CNT_PAYMENT,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_DAYS_TERMINATION,
MAX(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_TYPE_SUITE_null_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Group_of_people' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Group_of_people_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_A' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_A_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Other_B' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Other_B_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Children' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Children_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Spouse_partner' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Spouse_partner_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Family' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Family_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'Unaccompanied' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_Unaccompanied_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_AMT_ANNUITY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_AMT_APPLICATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_AMT_CREDIT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_DAYS_DECISION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_CNT_PAYMENT,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_DAYS_TERMINATION,
AVG(CASE WHEN NAME_TYPE_SUITE= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_TYPE_SUITE_null_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_House_Construction_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Animals_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Insurance_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Education_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Fitness_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medicine_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Tourism_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Other_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Gardening_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Vehicles_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Homewares_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_ANNUITY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_APPLICATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_CREDIT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_DECISION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_CNT_PAYMENT,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_TERMINATION,
SUM(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_House_Construction_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Animals_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Insurance_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Education_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Fitness_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medicine_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Tourism_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Other_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Gardening_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Vehicles_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Homewares_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_ANNUITY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_APPLICATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_CREDIT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_DECISION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_CNT_PAYMENT,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_TERMINATION,
MIN(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_House_Construction_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Animals_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Insurance_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Education_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Fitness_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medicine_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Tourism_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Other_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Gardening_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Vehicles_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Homewares_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_ANNUITY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_APPLICATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_CREDIT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_DECISION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_CNT_PAYMENT,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_TERMINATION,
MAX(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'House_Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_House_Construction_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Animals' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Animals_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Insurance' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Insurance_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Education' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Education_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Additional_Service' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Additional_Service_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Fitness' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Fitness_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Direct_Sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Direct_Sales_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medicine' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medicine_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Tourism' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Tourism_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Office_Appliances' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Office_Appliances_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Other' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Other_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Gardening' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Gardening_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Sport_and_Leisure' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Sport_and_Leisure_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Vehicles' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Vehicles_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Medical_Supplies' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Medical_Supplies_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Homewares' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Homewares_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Auto_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Auto_Accessories_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_ANNUITY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_APPLICATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_CREDIT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_DECISION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_CNT_PAYMENT,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_DAYS_TERMINATION,
AVG(CASE WHEN NAME_GOODS_CATEGORY= 'Clothing_and_Accessories' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_GOODS_CATEGORY_Clothing_and_Accessories_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cars_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cards_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PORTFOLIO_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PORTFOLIO_Cash_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_AMT_ANNUITY,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_AMT_APPLICATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_AMT_CREDIT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_DAYS_DECISION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_CNT_PAYMENT,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PORTFOLIO_POS_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_AMT_CREDIT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_DAYS_DECISION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_CNT_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cars_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_AMT_ANNUITY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_AMT_APPLICATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_AMT_CREDIT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_DAYS_DECISION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_CNT_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cards_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_AMT_ANNUITY,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_AMT_APPLICATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_AMT_CREDIT,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_DAYS_DECISION,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_CNT_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PORTFOLIO_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_AMT_ANNUITY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_AMT_APPLICATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_AMT_CREDIT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_DAYS_DECISION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_CNT_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PORTFOLIO_Cash_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_AMT_ANNUITY,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_AMT_APPLICATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_AMT_CREDIT,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_DAYS_DECISION,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_CNT_PAYMENT,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PORTFOLIO_POS_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_AMT_CREDIT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_DAYS_DECISION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_CNT_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cars_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_AMT_ANNUITY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_AMT_APPLICATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_AMT_CREDIT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_DAYS_DECISION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_CNT_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cards_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_AMT_ANNUITY,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_AMT_APPLICATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_AMT_CREDIT,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_DAYS_DECISION,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_CNT_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PORTFOLIO_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_AMT_ANNUITY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_AMT_APPLICATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_AMT_CREDIT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_DAYS_DECISION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_CNT_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PORTFOLIO_Cash_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_AMT_ANNUITY,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_AMT_APPLICATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_AMT_CREDIT,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_DAYS_DECISION,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_CNT_PAYMENT,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PORTFOLIO_POS_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cars' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cars_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cards' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cards_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PORTFOLIO_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PORTFOLIO_Cash_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_AMT_ANNUITY,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_AMT_APPLICATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_AMT_CREDIT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_DAYS_DECISION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_CNT_PAYMENT,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PORTFOLIO= 'POS' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PORTFOLIO_POS_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_DAYS_DECISION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_CNT_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_walk_in_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_DAYS_DECISION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_CNT_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_x_sell_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_PRODUCT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_AMT_ANNUITY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_AMT_APPLICATION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_AMT_CREDIT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_DAYS_DECISION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_CNT_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_walk_in_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_AMT_ANNUITY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_AMT_APPLICATION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_AMT_CREDIT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_DAYS_DECISION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_CNT_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_x_sell_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_AMT_ANNUITY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_AMT_APPLICATION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_AMT_CREDIT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_DAYS_DECISION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_CNT_PAYMENT,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_DAYS_TERMINATION,
MIN(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_PRODUCT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_AMT_ANNUITY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_AMT_APPLICATION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_AMT_CREDIT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_DAYS_DECISION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_CNT_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_walk_in_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_AMT_ANNUITY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_AMT_APPLICATION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_AMT_CREDIT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_DAYS_DECISION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_CNT_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_x_sell_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_AMT_ANNUITY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_AMT_APPLICATION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_AMT_CREDIT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_DAYS_DECISION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_CNT_PAYMENT,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_DAYS_TERMINATION,
MAX(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_PRODUCT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_AMT_ANNUITY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_AMT_APPLICATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_AMT_CREDIT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_DAYS_DECISION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_CNT_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'walk_in' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_walk_in_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_AMT_ANNUITY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_AMT_APPLICATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_AMT_CREDIT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_DAYS_DECISION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_CNT_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'x_sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_x_sell_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_PRODUCT_TYPE= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_PRODUCT_TYPE_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Car_dealer_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_AP_Cash_loan_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Contact_center_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Regional_Local_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Stone_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Country_wide_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_ANNUITY,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_APPLICATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_APPLICATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_CREDIT ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_CREDIT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_AMT_GOODS_PRICE,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_RATE_DOWN_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_DECISION ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_DECISION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_SELLERPLACE_AREA,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN CNT_PAYMENT ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_CNT_PAYMENT,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DRAWING,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_TERMINATION,
SUM(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Car_dealer_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_AP_Cash_loan_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Contact_center_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Regional_Local_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Stone_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Country_wide_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_AMT_ANNUITY,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_APPLICATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_AMT_APPLICATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_CREDIT ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_AMT_CREDIT,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_AMT_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_AMT_GOODS_PRICE,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_RATE_DOWN_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_DECISION ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_DECISION,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_SELLERPLACE_AREA,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN CNT_PAYMENT ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_CNT_PAYMENT,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DRAWING,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_TERMINATION,
MIN(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Car_dealer_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_AP_Cash_loan_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Contact_center_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Regional_Local_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Stone_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Country_wide_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_AMT_ANNUITY,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_APPLICATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_AMT_APPLICATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_CREDIT ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_AMT_CREDIT,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_AMT_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_AMT_GOODS_PRICE,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_RATE_DOWN_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_DECISION ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_DECISION,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_SELLERPLACE_AREA,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN CNT_PAYMENT ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_CNT_PAYMENT,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DRAWING,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_TERMINATION,
MAX(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Car_dealer' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Car_dealer_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Channel_of_corporate_sales' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Channel_of_corporate_sales_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'AP_Cash_loan' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_AP_Cash_loan_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Contact_center' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Contact_center_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Regional_Local' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Regional_Local_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Stone' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Stone_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Country_wide' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Country_wide_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_ANNUITY,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_APPLICATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_APPLICATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_CREDIT ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_CREDIT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_AMT_GOODS_PRICE,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_RATE_DOWN_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_DECISION ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_DECISION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_SELLERPLACE_AREA,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN CNT_PAYMENT ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_CNT_PAYMENT,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DRAWING,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_FIRST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_LAST_DUE,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_DAYS_TERMINATION,
AVG(CASE WHEN CHANNEL_TYPE= 'Credit_and_cash_offices' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_CHANNEL_TYPE_Credit_and_cash_offices_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Industry_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Clothing_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Construction_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Furniture_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_SELLER_INDUSTRY_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Industry_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Clothing_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Construction_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Furniture_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_AMT_ANNUITY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_AMT_APPLICATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_AMT_CREDIT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_DAYS_DECISION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_CNT_PAYMENT,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_DAYS_TERMINATION,
MIN(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_SELLER_INDUSTRY_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Industry_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Clothing_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Construction_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Furniture_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_AMT_ANNUITY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_AMT_APPLICATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_AMT_CREDIT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_DAYS_DECISION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_CNT_PAYMENT,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_DAYS_TERMINATION,
MAX(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_SELLER_INDUSTRY_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'MLM_partners' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_MLM_partners_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Jewelry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Jewelry_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Auto_technology' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Auto_technology_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Industry' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Industry_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Clothing' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Clothing_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Construction' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Construction_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Furniture' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Furniture_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Connectivity' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Connectivity_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'Consumer_electronics' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_Consumer_electronics_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_SELLER_INDUSTRY= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_SELLER_INDUSTRY_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_action_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_YIELD_GROUP_low_normal_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_YIELD_GROUP_high_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_YIELD_GROUP_middle_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_AMT_ANNUITY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_AMT_APPLICATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_AMT_CREDIT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_AMT_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_AMT_GOODS_PRICE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_RATE_DOWN_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_DAYS_DECISION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_SELLERPLACE_AREA,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_CNT_PAYMENT,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DRAWING,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_DAYS_TERMINATION,
SUM(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_NAME_YIELD_GROUP_XNA_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_DAYS_DECISION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_CNT_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_DAYS_TERMINATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_action_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_AMT_ANNUITY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_AMT_APPLICATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_AMT_CREDIT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_DAYS_DECISION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_CNT_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_DAYS_TERMINATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_YIELD_GROUP_low_normal_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_AMT_ANNUITY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_AMT_APPLICATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_AMT_CREDIT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_DAYS_DECISION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_CNT_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_DAYS_TERMINATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_YIELD_GROUP_high_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_AMT_ANNUITY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_AMT_APPLICATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_AMT_CREDIT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_DAYS_DECISION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_CNT_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_DAYS_TERMINATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_YIELD_GROUP_middle_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_AMT_ANNUITY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_AMT_APPLICATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_AMT_CREDIT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_AMT_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_AMT_GOODS_PRICE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_RATE_DOWN_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_DAYS_DECISION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_SELLERPLACE_AREA,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_CNT_PAYMENT,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DRAWING,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_DAYS_TERMINATION,
MIN(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_NAME_YIELD_GROUP_XNA_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_DAYS_DECISION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_CNT_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_DAYS_TERMINATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_action_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_AMT_ANNUITY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_AMT_APPLICATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_AMT_CREDIT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_DAYS_DECISION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_CNT_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_DAYS_TERMINATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_YIELD_GROUP_low_normal_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_AMT_ANNUITY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_AMT_APPLICATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_AMT_CREDIT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_DAYS_DECISION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_CNT_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_DAYS_TERMINATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_YIELD_GROUP_high_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_AMT_ANNUITY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_AMT_APPLICATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_AMT_CREDIT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_DAYS_DECISION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_CNT_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_DAYS_TERMINATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_YIELD_GROUP_middle_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_AMT_ANNUITY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_AMT_APPLICATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_AMT_CREDIT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_AMT_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_AMT_GOODS_PRICE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_RATE_DOWN_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_DAYS_DECISION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_SELLERPLACE_AREA,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_CNT_PAYMENT,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DRAWING,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_DAYS_TERMINATION,
MAX(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_NAME_YIELD_GROUP_XNA_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_action' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_action_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'low_normal' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_YIELD_GROUP_low_normal_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_YIELD_GROUP_high_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_YIELD_GROUP_middle_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_ANNUITY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_AMT_ANNUITY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_APPLICATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_AMT_APPLICATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_CREDIT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_AMT_CREDIT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_AMT_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_AMT_GOODS_PRICE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_RATE_DOWN_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_DECISION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_DAYS_DECISION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_SELLERPLACE_AREA,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN CNT_PAYMENT ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_CNT_PAYMENT,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DRAWING,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_DAYS_FIRST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_DAYS_LAST_DUE,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_DAYS_TERMINATION,
AVG(CASE WHEN NAME_YIELD_GROUP= 'XNA' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_NAME_YIELD_GROUP_XNA_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_null_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Card_Street_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_INSURED_ON_APPROVAL,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_AMT_ANNUITY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_AMT_APPLICATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_AMT_CREDIT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_AMT_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_AMT_GOODS_PRICE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_HOUR_APPR_PROCESS_START,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_NFLAG_LAST_APPL_IN_DAY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_RATE_DOWN_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIMARY,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIVILEGED,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_DAYS_DECISION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_SELLERPLACE_AREA,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_CNT_PAYMENT,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DRAWING,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE_1ST_VERSION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_DAYS_TERMINATION,
SUM(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS SUM_PRODUCT_COMBINATION_Cash_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_null_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Card_Street_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_INSURED_ON_APPROVAL,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_AMT_ANNUITY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_AMT_APPLICATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_AMT_CREDIT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_AMT_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_AMT_GOODS_PRICE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_HOUR_APPR_PROCESS_START,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_NFLAG_LAST_APPL_IN_DAY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_RATE_DOWN_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIMARY,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIVILEGED,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_DAYS_DECISION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_SELLERPLACE_AREA,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_CNT_PAYMENT,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DRAWING,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE_1ST_VERSION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_DAYS_TERMINATION,
MIN(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MIN_PRODUCT_COMBINATION_Cash_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_null_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Card_Street_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_INSURED_ON_APPROVAL,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_AMT_ANNUITY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_AMT_APPLICATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_AMT_CREDIT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_AMT_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_AMT_GOODS_PRICE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_HOUR_APPR_PROCESS_START,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_NFLAG_LAST_APPL_IN_DAY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_RATE_DOWN_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIMARY,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIVILEGED,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_DAYS_DECISION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_SELLERPLACE_AREA,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_CNT_PAYMENT,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DRAWING,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE_1ST_VERSION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_DAYS_TERMINATION,
MAX(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS MAX_PRODUCT_COMBINATION_Cash_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'null' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_null_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_others_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_others_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_other_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_other_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_low_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_middle_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_high_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_Street_high' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_Street_high_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_X_Sell' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_X_Sell_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_without_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_without_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_industry_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_industry_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Card_Street' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Card_Street_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_low' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_low_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash_X_Sell_middle' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_X_Sell_middle_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_mobile_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_mobile_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'POS_household_with_interest' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_POS_household_with_interest_NFLAG_INSURED_ON_APPROVAL,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_ANNUITY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_AMT_ANNUITY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_APPLICATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_AMT_APPLICATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_CREDIT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_AMT_CREDIT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_AMT_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN AMT_GOODS_PRICE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_AMT_GOODS_PRICE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN HOUR_APPR_PROCESS_START ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_HOUR_APPR_PROCESS_START,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_LAST_APPL_IN_DAY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_NFLAG_LAST_APPL_IN_DAY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_DOWN_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_RATE_DOWN_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIMARY ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIMARY,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN RATE_INTEREST_PRIVILEGED ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_RATE_INTEREST_PRIVILEGED,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_DECISION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_DAYS_DECISION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN SELLERPLACE_AREA ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_SELLERPLACE_AREA,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN CNT_PAYMENT ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_CNT_PAYMENT,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DRAWING ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DRAWING,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_FIRST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_DAYS_FIRST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE_1ST_VERSION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE_1ST_VERSION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_LAST_DUE ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_DAYS_LAST_DUE,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN DAYS_TERMINATION ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_DAYS_TERMINATION,
AVG(CASE WHEN PRODUCT_COMBINATION= 'Cash' THEN NFLAG_INSURED_ON_APPROVAL ELSE 0 END) AS AVG_PRODUCT_COMBINATION_Cash_NFLAG_INSURED_ON_APPROVAL
FROM
  df_previous_application
GROUP BY
  SK_ID_CURR
""")


In [115]:
df_previous_application_contract_num_and_cat.createOrReplaceTempView("df_previous_application_contract_num_and_cat")

##Analisando quantidade de novas variáveis explicativas nós conseguimos criar após as etapas anteriores

In [116]:
# Conta o número de colunas na tabela
num_colunas_base = len(df_previous_application.columns)
num_colunas_base_cat = len(df_previous_application_cat.columns)
num_colunas_base_num = len(df_previous_application_num.columns)
num_colunas_base_cat_e_num = len(df_previous_application_contract_num_and_cat.columns)

# Imprime o resultado
print("A base original previous_application", num_colunas_base, "colunas.")
print('')
print("A base categórica da previous_application possui", num_colunas_base_cat, " novas colunas.")
print('')
print("A base numérica da previous_application possui", num_colunas_base_num, " novas colunas.")
print('')
print("A base numérica com categórica da previous_application possui", num_colunas_base_cat_e_num, " novas colunas.")
print('')

A base original previous_application 37 colunas.

A base categórica da previous_application possui 521  novas colunas.

A base numérica da previous_application possui 77  novas colunas.

A base numérica com categórica da previous_application possui 8057  novas colunas.



In [117]:
df_application_train = spark.read.csv("/content/gdrive/MyDrive/HD/CURSOS/Ciência - Análise de Dados/Pod Academy/Formação em Big Data e Analytics/Hackathon - Ciência de Dados/11 - Hackaton/dados/application_train.csv",sep=',',header=True, encoding='utf-8', inferSchema=True)
df_application_train.createOrReplaceTempView("df_application_train")

In [118]:
# Conta o número de colunas na tabela
num_colunas_base = len(df_application_train.columns)

# Imprime o resultado
print("A base original application_train", num_colunas_base, "colunas.")
print('')

A base original application_train 172 colunas.



In [119]:
df_application_train.count()

215257

Além do dataframe df_application_train que tem 172 colunas e do df_previous_application que tem 37 colunas, criamos mais 3 novos dataframes utilizando agregações e regras de negócio, onde:

- df_previous_application_cat: 521 colunas
- df_previous_application_num: 77 colunas
- df_previous_application_contract_num_and_cat: 2281 colunas

Lembrando que para todos os dataframes temos que o número de variáveis é igual ao número de colunas - 1, pois a coluna 'SK_ID_CURR' é a chave de todos os dataframes.

Inicialmente tínhamos 171 variáveis para a tabela df_application_train e 36 variáveis para o df_previous_application, que totalizam 207 variáveis.

###Juntando ao df_application_train os 3 dataframes criados usando a coluna SK_ID_CURR

In [120]:
df_previous_application_junto = df_application_train.join(df_previous_application_cat, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_num, "SK_ID_CURR",  how = 'left')\
               .join(df_previous_application_contract_num_and_cat, "SK_ID_CURR",  how = 'left')

In [121]:
df_previous_application_junto.count()

215257

###Calculando o número de colunas do dataframe df_junto

In [122]:
# Contando o número de colunas do df_previous_application_junto
num_colunas = len(df_previous_application_junto.columns)

# Imprimindo o resultado
print(num_colunas, "colunas.")

8824 colunas.


In [123]:
df_previous_application_junto.createOrReplaceTempView("df_previous_application_junto")

#Conclusão

Então, temos o dataframe final df_junto com 3048 variáveis. De 207 variáveis iniciais passamos para 3048 variáveis, ou seja, aumentamos o número de variáveis em aproximandamente 14 vezes.